End to end test of our general network structure.  Will test on the known non-linear system $$\dot{x} = ax,$$ $$\dot{y} = b(x-x^2).$$  The Koopman operator for this system is $$\frac{d}{dt}\begin{bmatrix}z_1\\ z_2\\ z_3\end{bmatrix} = \begin{bmatrix} a & 0 & 0\\ b& 0 & -b\\ 0 & 0 & 2a\end{bmatrix}\begin{bmatrix}z_1\\ z_2\\ z_3\end{bmatrix}$$ where $$z_1 = x,$$ $$z_2 = y,$$ $$z_3 = x^2.$$ 

We will have to select $a$ and $b$ ahead of time and train for specific instances of them (equivilent to switching hamiltionians).

The solution to the original system for us to compare results agains is $$x(t) = e^{at}$$

In [2]:
from contextlib import redirect_stdout  #Used for writing model architecture to datafiles
import matplotlib.pyplot as plt         
from datetime import date               #Used for datafiles
import tensorflow as tf
import numpy as np
import scipy.integrate
import os


#Some GPU configuration
#Always uses the 1st GPU avalible (if avalible) unless 1st line is uncommented, in which case no GPU is used

#tf.config.set_visible_devices([], 'GPU') #uncomment to set tensorflow to use CPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) != 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

### $\phi$ and $\phi^{-1}$

In [3]:
STATE_DIMENSION = 3
ANTIKOOPMAN_DIMENSION = 2

#Train autoencoder on uniform distribution between -MAX_DIM_VALUE and MAX_DIM_VALUE
MAX_DIM_VALUE = 100.

#LecunUniform seems to give the most consistent results, 
#though the model still struggles to get started sometimes
INITILIZER = tf.keras.initializers.LecunUniform()


In [4]:
def generate_autoencoder_data(batch_size = 512):
    '''Generator for data when training the 
    autoencoder end-to-end
    '''
    koopman_dimension = 3
    triples = np.empty([batch_size, koopman_dimension])
    while True:
        for i in range(batch_size):
            z1, z2 = np.random.uniform(low=-MAX_DIM_VALUE, high=MAX_DIM_VALUE, size=2)
            z3 = z1*z1
            triples[i] = np.array([z1,z2,z3])

        yield (triples, triples)
        
def generate_encoder_data(batch_size = 4096):
    '''Generator to be used when training just
    the encoder (phi) on its own
    '''
    koopman_dimension = 3
    doubles = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    triples = np.empty([batch_size, koopman_dimension])
    while True:
        for i in range(batch_size):
            z1, z2 = np.random.uniform(low=-MAX_DIM_VALUE, high=MAX_DIM_VALUE, size=2)
            z3 = z1*z1
            doubles[i] = np.array([z1,z2])
            triples[i] = np.array([z1, z2, z3])
        yield(triples, doubles)
        
def generate_decoder_data(batch_size = 4096):
    '''Generator to be used when training just the
    decoder (phi_inv) on its own
    '''
    doubles = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    triples = np.empty([batch_size, STATE_DIMENSION])
    while True:
        for i in range(batch_size):
            z1, z2 = np.random.uniform(low=-MAX_DIM_VALUE, high=MAX_DIM_VALUE, size=2)
            z3 = z1*z1
            doubles[i] = np.array([z1,z2])
            triples[i]=np.array([z1,z2,z3])
        yield(doubles, triples)
        

In [5]:
#Input layers for the encoder and decoder, respectivley
initial_state = tf.keras.Input(shape = STATE_DIMENSION)
antikoop_state = tf.keras.Input(shape = ANTIKOOPMAN_DIMENSION)

##########################################ENCODER####################################################################
encoding_layer_1 = tf.keras.layers.Dense(8, activation="selu", name='encoding_layer_1', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(initial_state)
encoding_layer_2 = tf.keras.layers.Dense(32, activation="selu", name='encoding_layer_2', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(encoding_layer_1)
norm_layer_1 = tf.keras.layers.BatchNormalization(name='norm_layer_1')(encoding_layer_2)
encoding_layer_3 = tf.keras.layers.Dense(64, activation="selu", name='encoding_layer_3', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(norm_layer_1)
norm_layer_2 = tf.keras.layers.BatchNormalization(name='norm_layer_2')(encoding_layer_3)
encoding_layer_4 = tf.keras.layers.Dense(32, activation="selu", name='encoding_layer_4', kernel_initializer=INITILIZER, bias_initializer=INITILIZER)(norm_layer_2)
norm_layer_3 = tf.keras.layers.BatchNormalization(name='norm_layer_3')(encoding_layer_4)
encoding_layer_5 = tf.keras.layers.Dense(8, activation="selu", name='encoding_layer_5', kernel_initializer=INITILIZER, bias_initializer=INITILIZER)(norm_layer_3)
encoded_state = tf.keras.layers.Dense(ANTIKOOPMAN_DIMENSION, activation="selu", name='bottleneck', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(encoding_layer_5)
#####################################################################################################################

#########################################DECODER#####################################################################
decoding_layer_1 = tf.keras.layers.Dense(16, activation = "selu", name='decoding_layer_1', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(antikoop_state)
decoding_layer_2 = tf.keras.layers.Dense(32, activation = "selu", name='decoding_layer_2', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(decoding_layer_1)
norm_layer_4 = tf.keras.layers.BatchNormalization(name='norm_layer_4')(decoding_layer_2)
decoding_layer_3 = tf.keras.layers.Dense(64, activation = "selu", name='decoding_layer_3', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(norm_layer_4)
norm_layer_5 = tf.keras.layers.BatchNormalization(name='norm_layer_5')(decoding_layer_3)
decoding_layer_4 = tf.keras.layers.Dense(32, activation = "selu", name='decoding_layer_4', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(norm_layer_5)
norm_layer_6 = tf.keras.layers.BatchNormalization(name='norm_layer_6')(decoding_layer_4)
decoding_layer_5 = tf.keras.layers.Dense(16, activation = "selu", name='decoding_layer_5', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(norm_layer_6)
decoded_state = tf.keras.layers.Dense(STATE_DIMENSION, activation = "selu", name='decoded_layer', kernel_initializer = INITILIZER, bias_initializer = INITILIZER)(decoding_layer_5)
#####################################################################################################################



#Model declarations
#Autoencoder = tf.keras.Model(initial_state, decoded_state, name = 'Autoencoder')

Phi = tf.keras.Model(inputs=initial_state, outputs = encoded_state, name='Phi')
Phi_inv = tf.keras.Model(inputs = antikoop_state, outputs = decoded_state, name='Phi_inv')

Autoencoder = tf.keras.models.Sequential([Phi, Phi_inv], name='Autoencoder')


In [6]:
def autoencoding_loss(y_true, y_pred):
    '''The L2 norm of the input vector
    and the autoencoded vector
    Scale down to make gradients more tractable
    with large vectors?'''
    
    return tf.norm((y_true-y_pred)*tf.constant([1.2,1.2,0.8]), ord=2)
    
    #return tf.abs(y_true[0]-y_pred[0])/y_true[0] + tf.abs(y_true[1]-y_pred[1])/y_true[1] + tf.abs(y_true[2]-y_pred[2])/y_true[2]
    
    #return tf.abs(y_pred[0]*y_pred[0] - y_pred[2]) + tf.abs(y_true[0]-y_pred[0]) + tf.abs(y_true[1] - y_pred[1]) + tf.abs(y_true[2]-y_pred[2])

def encoder_loss(y_true, y_pred):
    return tf.norm(y_true-y_pred, ord=2)

def decoder_loss(y_true, y_pred):
    return tf.norm(y_true-y_pred, ord=2) + 2*tf.abs(y_true[0]-y_pred[0]) + 2*tf.abs(y_true[1]-y_pred[1])



def predict_single_state(state, encoder = Phi, decoder = Phi_inv):
    '''Outputs the prediction of a single 
    state.  Primarily for sanity checks.
    '''
    encoded = encoder(np.array([state,]))
    decoded = decoder(encoder(np.array([state,]))).numpy()[0]
    L2_dist = np.linalg.norm(state-decoded, ord=2)


    print('Initial State:{}\nEncoded State:{}\nDecoded State:{}\nL2 Distance:{}\nInput z1 Squared:{}\nOutput z1 Squared:{}\nz3-z1^2:{}\nLoss:{}'.format(
            state, encoded.numpy(), decoded, L2_dist, state[0]*state[0],
            decoded[0]*decoded[0], decoded[2]-decoded[0]*decoded[0], 
            L2_dist + np.abs(decoded[2]-decoded[0]*decoded[0])))
          
    return None


###################DATA WRITING FUNCTIONS#####################

##############################################################
def write_history(history, model, loss = 'autoencoding_loss', 
                  optimizer='Adam', lr='.001', 
                  batch_size='1024', datadir='./datafiles/'):
    '''Writes training history to a datafile
    This will create a new trial datafile.  If the model has 
    had additional training, append_history should be used instead.
    
    PARAMS:
    -------
    history - The history callback returned by the model.fit method in keras
    model - The model (or list of models) which we want to write the architecture of to the datafile
    string loss - The loss function the model was trained on
    string optimizer - The optimizer the model was compiled with
    string lr - The learning rate the model was initially compiled with
    string spe - The number of steps per epoch
    string batch_size - The number of samples trained on per step
    string datadir - Directory where the datafiles are stored
    '''
    
    rundatadir = datadir
    filename = 'trial'+str(len(os.listdir(rundatadir)))

    with open(rundatadir+filename+'.data', 'w') as f:
        f.write(str(date.today())+'\n')
        for key in history.history.keys():
            f.write(key+',')
            for epoch in range(history.params['epochs']):
                f.write(str(history.history[key][epoch])+',')
            f.write('\n')
        f.write("Loss,{}\nOptimizer,{}\nLearning Rate,{}\nSteps Per Epoch,{}\nBatch Size,{}\nEpochs,{}\n".format(loss,optimizer,lr,history.params['steps'],batch_size, history.params['epochs']))
        f.write('\n')
        with redirect_stdout(f):
            if type(model) == list:
                for i in model:              
                    i.summary()
            else:
                model.summary()
    return rundatadir+filename+'.data'

#####################################################################
#####################################################################

def append_history(history, trial, datadir='./datafiles/', params_update = True, 
                   params = {'Loss':None, 'Optimizer':None, 'Learning Rate':None, 'Batch Size':None}):
    '''Appends new training data to trial datafile.
    This will only work with datafiles written using write_history (or files of identical form)
    
    PARAMS:
    -------
    history - The history callback containing the new run's data
    int trial - The trial number to append the data to.  If we want to append data to 
            trial43.data, this would be 43
    str datadir - The directory containing the datafile
    bool params_update - Boolean indicating if we should update parameters (loss used, optimizer, etc.)
                    in addition to adding the new loss data
    dict params - Dictionary containing updated parameter values
                  If parameter is not included, the previous value 
                  written for that parameter will be repeated
    '''
    
    filename = 'trial'+str(trial)+'.data'
    
    
    newlines = []
    keys = history.history.keys()
    
    
    for k in ['Loss', 'Optimizer', 'Learning Rate', 'Batch Size']:
        if k not in params.keys():
            params[k] = None
    params['Epochs'] = history.params['epochs']
    params['Steps Per Epoch'] = history.params['steps']
   
      
    #Read old data and add in new data as it is read
    with open(datadir+filename, 'r') as f:
        for line in f.readlines():
            
            tag = line.split(',')[0]
            
            if tag in keys:
                newdata = [str(x) for x in history.history[tag]]
                newlines.append(line.split(',')[:-1] + newdata ) #[:-1] to drop the newline
            elif (tag in params.keys() and params_update == True):
                if params[tag] is None:
                    newlines.append(line.strip().split(',')[:] + [line.strip().split(',')[-1]])
                else:
                    newdata = [str(params[tag])]
                    newlines.append(line.strip().split(',')[:] + newdata)
            else:
                newlines.append(line)
    
    #Write the old data with the appended data
    with open(datadir+filename, 'w') as f:
        for el in newlines:
            if type(el) == list:
                f.write(','.join(el))
                f.write('\n')
            else:
                f.write(el)
    return
    
    
#####################################################################
#####################################################################
    
def loss_plot(trial, datadir='./datafiles/', savefig = True,
              figdir = './figures/', logplot=False,
              metric = None, mark_runs = False, 
              skip_epochs=0, mark_lowest = True):
    '''Creates a plot of the loss/metric for the given trial number
    '''
    
    if metric == None:
        metric = 'loss'

    losses = []
    runs = []
    
    #Read in the data
    with open(datadir+'trial'+str(trial)+'.data', 'r') as f:
        for line in f.readlines():
            if line.split(',')[0] == metric:
                losses = [float(x) for x in line.strip().split(',')[1:]]
                if not mark_runs:
                    break
            elif (line.split(',')[0] == 'Epochs' and mark_runs == True):
                runs = ['0.']+line.strip().split(',')[1:]
                runs = [float(runs[i-1])+float(runs[i-2]) for i in range(2, len(runs))]
                break
            
    
    fig, ax = plt.subplots(1,1, figsize = (8,8))


    ax.plot(range(len(losses[skip_epochs:])), losses[skip_epochs:])
    if mark_runs:
        for i in runs:
            ax.plot([i,i], ax.get_ylim(), c='black', ls=':')
    if mark_lowest:
        lowest = min(losses)
        ax.plot(losses.index(lowest), lowest, 'go')
 #       ax.text(ax.get_xlim()[1]-0.05*ax.get_xlim()[0], ax.get_ylim()[1]-0.05*ax.get_ylim()[0], 'Lowest loss: {}'.format(lowest))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    if logplot:
        ax.set_yscale('log')
    ax.set_title('Trial '+str(trial)+' Loss')

    if savefig:
        fig.savefig(figdir+'trial{}.png'.format(trial))
    
    return None

#####################################################################
#####################################################################

In [7]:
Autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = .0001), loss=autoencoding_loss, metrics = ['mse', 'mae'], run_eagerly=False)

In [ ]:
history = Autoencoder.fit(generate_autoencoder_data(4096), steps_per_epoch=50,epochs=100)

In [40]:
z1, z2 = np.random.uniform(low=-MAX_DIM_VALUE, high=MAX_DIM_VALUE, size=2)
z3 = z1*z1

In [41]:
predict_single_state([z1,z2,z3])

Initial State:[-6.090724688422071, -8.191452779735542, 37.09692723015414]
Encoded State:[[-1.3374066  -0.22778395]]
Decoded State:[-1.5950345 -1.7481171 -1.7580993]
L2 Distance:39.64140377549842
Input z1 Squared:37.09692723015414
Output z1 Squared:2.544135093688965
z3-z1^2:-4.302234649658203
Loss:43.94363842515662


In [74]:
write_history(history, [Phi, Phi_inv, Autoencoder], datadir='./trials/', batch_size=4096)

'./trials/trial0.data'

In [136]:
append_history(history, 2, datadir='./trials/', params={'Learning Rate':.0001})

In [128]:
Phi.save('./models/Phi.h5')

### Nonlinear evolution function

In [8]:
#Configure and set up koopman operator
#These parameters keep z1 and z2 within +/-100 from t=0.0 to t=10.0 provided z1 and z2 are between 0 and 1 at t=0.0
system_a = 0.2
system_b = 0.1
koopman_operator = np.array([[system_a, 0, 0], [system_b, 0, -system_b], [0, 0, 2*system_a]])

In [13]:
def koopman_evolve(t, y):
    return np.matmul(koopman_operator, y)

        
def generate_dynamical_system_data(batch_size = 128, t_span = 10.0):
    koopman_dimension = 3
    systems = np.empty([batch_size, koopman_dimension])
    initial_conditions = np.empty([batch_size, koopman_dimension])
    while True:
        for i in range(batch_size):
            y0 = np.random.random(3)
            evolved = []
            evolution = scipy.integrate.solve_ivp(koopman_evolve, y0, (0, t_span)).y
            for k in evolution:
                evolved.append(k[-1])
            systems[i] = evolved
            initial_conditions[i] = y0
        
        yield (initial_conditions, systems)
         
    
    
#Idealized phi and phi_inv, gives an upper bound on accuracy of 
#nonlinear network
def ideal_compression(state):
    return np.array([state[0], state[1]])

def ideal_decompression(state):
    return np.array([state[0], state[1], state[0]*state[0]])

#Generator which pre-applies perfect compression and decompression on system
def generate_ideal_system_data(batch_size = 128, t_span=10.0):
    systems = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    initial_conditions = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    while True:
        for i in range(batch_size):
            y0 = np.empty([STATE_DIMENSION])
            y0[0], y0[1] = np.random.random(2)
            y0[2] = y0[0]*y0[0]
            #y0.reshape(3,1)
            evolution = scipy.integrate.solve_ivp(koopman_evolve,(0,t_span), y0).y[:,-1]
            initial_conditions[i] = ideal_compression(y0)
            systems[i] = ideal_compression(evolution)
        yield(initial_conditions, systems)

def nonlinear_loss(y_true, y_pred):
    return tf.norm(y_true-y_pred, ord=2)


In [10]:
inputs = tf.keras.layers.Input(ANTIKOOPMAN_DIMENSION)

nonlinear_layer_1 = tf.keras.layers.Dense(8, activation='selu', name='nonlinea_layer_1')(inputs)
nonlinear_layer_2 = tf.keras.layers.Dense(16, activation='selu', name='nonlinear_layer_2')(nonlinear_layer_1)
nonlinear_layer_3 = tf.keras.layers.Dense(32, activation='selu', name='nonlinear_layer_3')(nonlinear_layer_2)
nonlinear_layer_4 = tf.keras.layers.Dense(16, activation='selu', name='nonlinear_layer_4')(nonlinear_layer_3)
nonlinear_layer_5 = tf.keras.layers.Dense(8, activation='selu', name='nonlinear_layer_5')(nonlinear_layer_4)
evolved = tf.keras.layers.Dense(ANTIKOOPMAN_DIMENSION, activation='selu', name='evolved_state_layer')(nonlinear_layer_5)

NonlinearF = tf.keras.models.Model(inputs=inputs, outputs=evolved)

In [16]:
NonlinearF.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001), loss=nonlinear_loss)

In [ ]:
history = NonlinearF.fit(generate_ideal_system_data(256), epochs=100, steps_per_epoch=50)

## Derivation of the Koopman Operator

We start with $$\dot{x} = ax,$$ $$\dot{y} = b(x-x^2).$$  We start by converting our existing variables, $x$ and $y$, into $z$'s.  This gives
$$z_1 = x\qquad \dot{z_1} = \dot{x} = ax = az_1$$
and
$$z_2 = y\qquad \dot{z_2} = \dot{y} = b(x-x^2) = b(z_1-z_3)$$
Here, since $x^2$ is a nonlinear term, we had to introduce a linear term to replace it.  This leads to
$$z_3 = x^2\qquad \dot{z_3} = 2x\dot{x} = 2ax^2 = 2az_3$$
All of our original variables have been accounted for, and all of our $z$'s have their derivates described using only linear terms.  By simply putting this together into a matrix, we have our Koopman operator.